# 🔧 Week 12: Fine-Tuning LLMs

This notebook covers fine-tuning techniques for large language models.

## Table of Contents
1. [Fine-Tuning Fundamentals](#1-fine-tuning-fundamentals)
2. [Full Fine-Tuning](#2-full-fine-tuning)
3. [Parameter-Efficient Methods](#3-parameter-efficient-methods)
4. [LoRA Deep Dive](#4-lora-deep-dive)
5. [Training Best Practices](#5-training-best-practices)
6. [Evaluation and Iteration](#6-evaluation-and-iteration)

---

## 1. Fine-Tuning Fundamentals

### 1.1 Why Fine-Tune?

| Approach | When to Use | Example |
|----------|-------------|--------|
| **Prompting** | Simple tasks, no data | Zero-shot classification |
| **Few-shot** | Limited examples | In-context learning |
| **Fine-tuning** | Specific domain/task | Legal document Q&A |

### 1.2 Types of Fine-Tuning

```
┌─────────────────────────────────────────────────┐
│           FINE-TUNING SPECTRUM                  │
├─────────────┬─────────────┬────────────────────┤
│  Full       │   PEFT      │   Prompting        │
│  Fine-Tune  │   (LoRA)    │   Only             │
├─────────────┼─────────────┼────────────────────┤
│ All params  │ Few params  │ No training        │
│ Most data   │ Less data   │ Minimal data       │
│ Best perf   │ Good perf   │ Quick iteration    │
└─────────────┴─────────────┴────────────────────┘
```

In [ ]:
# Example training data format
training_examples = [
    {
        "instruction": "Summarize the following text.",
        "input": "Machine learning is a subset of artificial intelligence...",
        "output": "ML is an AI subfield that enables learning from data."
    },
    {
        "instruction": "Classify the sentiment of this review.",
        "input": "This product exceeded my expectations!",
        "output": "Positive"
    }
]

print("Training data format:")
for i, ex in enumerate(training_examples):
    print(f"\nExample {i+1}:")
    print(f"  Instruction: {ex['instruction']}")
    print(f"  Input: {ex['input'][:40]}...")
    print(f"  Output: {ex['output']}")

---

## 2. Full Fine-Tuning

### 2.1 When to Use Full Fine-Tuning

✅ **Good for:**
- Large datasets (10K+ examples)
- Maximum performance required
- Significant domain shift

❌ **Drawbacks:**
- Requires significant compute
- Risk of catastrophic forgetting
- Full model storage needed

In [ ]:
# Pseudocode for full fine-tuning
print("""
Full Fine-Tuning Pipeline:
==========================

1. Load pre-trained model
   model = AutoModelForCausalLM.from_pretrained("base-model")
   
2. Prepare dataset
   dataset = load_and_format_data()
   
3. Configure training
   training_args = TrainingArguments(
       learning_rate=2e-5,
       num_train_epochs=3,
       per_device_train_batch_size=4,
       gradient_accumulation_steps=4,
   )
   
4. Train
   trainer = Trainer(model=model, args=training_args, train_dataset=dataset)
   trainer.train()
   
5. Save
   model.save_pretrained("fine-tuned-model")
""")

---

## 3. Parameter-Efficient Methods

### 3.1 PEFT Overview

| Method | Trainable Params | Memory | Quality |
|--------|-----------------|--------|--------|
| **LoRA** | ~0.1% | Low | High |
| **QLoRA** | ~0.1% | Very Low | High |
| **Prefix Tuning** | ~1% | Low | Medium |
| **Prompt Tuning** | <0.1% | Very Low | Medium |
| **Adapters** | ~1-3% | Low | High |

In [ ]:
# Comparison of trainable parameters
model_params = 7_000_000_000  # 7B model

methods = {
    "Full Fine-Tune": 1.0,
    "Adapters": 0.03,
    "Prefix Tuning": 0.01,
    "LoRA": 0.001,
    "Prompt Tuning": 0.0001,
}

print("Trainable Parameters Comparison (7B model):")
print("=" * 50)

for method, ratio in methods.items():
    params = model_params * ratio
    print(f"{method:20s}: {params:>15,.0f} params ({ratio:.4%})")

---

## 4. LoRA Deep Dive

### 4.1 How LoRA Works

LoRA (Low-Rank Adaptation) adds trainable low-rank matrices:

$$W' = W + \Delta W = W + BA$$

Where:
- $W$ = Original frozen weights (d × k)
- $B$ = Trainable matrix (d × r)
- $A$ = Trainable matrix (r × k)
- $r$ = Rank (typically 4-64)

```
Original: [d × k] weights = d * k parameters
LoRA:     [d × r] + [r × k] = d*r + r*k parameters

If d=4096, k=4096, r=8:
  Original: 16,777,216 params
  LoRA:     32,768 + 32,768 = 65,536 params (0.4%)
```

In [ ]:
import numpy as np

class LoRALayer:
    """
    LoRA (Low-Rank Adaptation) layer implementation.
    
    Adds trainable low-rank matrices to frozen weights.
    """
    
    def __init__(self, d: int, k: int, r: int = 8, alpha: float = 16):
        """
        Args:
            d: Input dimension
            k: Output dimension
            r: LoRA rank
            alpha: Scaling factor
        """
        self.d = d
        self.k = k
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r
        
        # Initialize LoRA matrices
        # A: Gaussian init, B: Zero init
        self.A = np.random.randn(r, k) * 0.01
        self.B = np.zeros((d, r))
        
        # Simulated frozen weights
        self.W_frozen = np.random.randn(d, k) * 0.1
    
    def forward(self, x):
        """Forward pass with LoRA."""
        # Original path
        h = x @ self.W_frozen.T
        
        # LoRA path: x -> A -> B
        lora_out = (x @ self.A.T) @ self.B.T * self.scaling
        
        return h + lora_out
    
    @property
    def trainable_params(self):
        return self.d * self.r + self.r * self.k
    
    @property
    def total_params(self):
        return self.d * self.k

# Example
lora = LoRALayer(d=4096, k=4096, r=8)

print(f"Original params:  {lora.total_params:,}")
print(f"LoRA params:      {lora.trainable_params:,}")
print(f"Reduction:        {lora.trainable_params / lora.total_params:.4%}")

### 4.2 LoRA Configuration

| Parameter | Typical Value | Effect |
|-----------|--------------|--------|
| **r (rank)** | 8-64 | Higher = more capacity |
| **alpha** | 16-32 | Scaling factor |
| **target_modules** | q_proj, v_proj | Which layers to adapt |
| **dropout** | 0.05-0.1 | Regularization |

In [ ]:
# LoRA Configuration Example
lora_config = {
    "r": 8,
    "lora_alpha": 16,
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"],
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM"
}

print("LoRA Configuration:")
for key, value in lora_config.items():
    print(f"  {key}: {value}")

---

## 5. Training Best Practices

### 5.1 Hyperparameters

In [ ]:
# Recommended hyperparameters for LoRA fine-tuning
training_config = {
    # Learning rate
    "learning_rate": 2e-4,  # Higher than full fine-tuning
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.03,
    
    # Batch size
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "effective_batch_size": 16,  # = 4 * 4
    
    # Training duration
    "num_train_epochs": 3,
    "max_steps": -1,  # Or specify max steps
    
    # Memory optimization
    "fp16": True,  # Or bf16 if supported
    "gradient_checkpointing": True,
    
    # Regularization
    "weight_decay": 0.01,
    "max_grad_norm": 1.0,
}

print("Training Configuration:")
print("=" * 40)
for key, value in training_config.items():
    print(f"{key:30s}: {value}")

### 5.2 Data Quality Tips

| Aspect | Recommendation |
|--------|---------------|
| **Quantity** | 1K-100K examples typical |
| **Quality** | Clean, consistent, diverse |
| **Format** | Match desired output format |
| **Balance** | Diverse task distribution |
| **Validation** | 10-20% held out |

---

## 6. Evaluation and Iteration

### 6.1 Evaluation Metrics

| Task | Metrics |
|------|--------|
| **Classification** | Accuracy, F1 |
| **Generation** | BLEU, ROUGE, Human eval |
| **QA** | Exact match, F1 |
| **Summarization** | ROUGE-L, factuality |

In [ ]:
# Simple evaluation loop
def evaluate_model(model, eval_data):
    """Evaluate fine-tuned model."""
    results = {
        "correct": 0,
        "total": len(eval_data),
        "examples": []
    }
    
    for example in eval_data:
        # Generate prediction
        prediction = "model.generate(example['input'])"
        
        # Check correctness
        is_correct = prediction == example["expected"]
        if is_correct:
            results["correct"] += 1
        
        results["examples"].append({
            "input": example["input"],
            "expected": example["expected"],
            "predicted": prediction,
            "correct": is_correct
        })
    
    results["accuracy"] = results["correct"] / results["total"]
    return results

print("Evaluation function defined!")

---

## 📝 Summary

### Key Takeaways

1. **Choose method based on resources**: Full fine-tuning for max performance, LoRA for efficiency
2. **LoRA is usually sufficient**: 90%+ of full fine-tuning quality at <1% params
3. **Data quality > quantity**: Clean, diverse data is crucial
4. **Iterate quickly**: Start small, validate, scale up

### Decision Framework

```
Do you have 10K+ examples? ──Yes──> Full Fine-Tuning
         │ No
         ▼
Limited compute? ──Yes──> QLoRA
         │ No
         ▼
       LoRA
```